## tracking the detected pupil



In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(precision=2, suppress=True)
# define plots to be inserted interactively
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
from LeCheapEyeTracker import Server, Client


## and now... let's calibrate

we grab 42 frames. why 42? I recommend to not move the head (hold them in your hands) and to look at the led of your webcam. when it's lit, move your eyes evenly on the screen.

In [2]:
%%writefile calibration_fixation.py
import cv2
import time
from LeCheapEyeTracker import Server, Client
from vispy import app
import numpy as np

N_frame = 42
et = Server()
img0 = et.cam.grab()
def stim(t):
    img0 = et.cam.grab()
    H, W, three = img0.shape
    img = img0.copy()
    img = cv2.circle(img, (W//2, H//2), 12, (0,0,255), -1)
    return img

T, fps = 7., 30
timeline = np.linspace(0, T, T*fps)
screen = Client(et, timeline)
app.run()
et.close()

Overwriting calibration_fixation.py


In [3]:
%run calibration_fixation.py

window size :  (1920, 1200)
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position


/usr/local/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [4]:
%%writefile calibration_horizontal.py
import cv2
import time
from LeCheapEyeTracker import Client, Server
from vispy import app
import numpy as np

et = Server()
T, fps = 7., 30
timeline = np.linspace(0, T, T*fps)

screen = Client(et, timeline)
app.run()
et.close()
print(screen.et.eye_pos)

Overwriting calibration_horizontal.py


In [5]:
%run calibration_horizontal.py

window size :  (1920, 1200)
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
[[(149.0, 60.0), 810605.426279076], [(155.0, 62.0), 810605.831643347]]


In [ ]:

pos_true = []
timeline = []
pos_h, pos_w = [], []
t_max = screen.et.eye_pos[-1][1]
t_min = screen.et.eye_pos[0][1]
print(t_min, t_max)
for pos, t0 in screen.et.eye_pos:
    pos_true.append(np.sin(2*np.pi*(t0-t_min)/(t_max-t_min)))
    pos_w.append(pos[0])
    pos_h.append(pos[1]) 
    timeline.append(t0-t_min)

pos_true = np.array(pos_true)
print(pos_true)
pos_true /= pos_true.max()

pos_w = np.array(pos_w)
pos_w -= pos_w.mean()
pos_w /= pos_w.max()

pos_h = np.array(pos_h)
pos_h -= pos_h.mean()
pos_h /= pos_h.max()

fig, axs = plt.subplots(1, 1, figsize=(15, 7.5))
axs.plot(timeline, pos_true, 'k')
axs.plot(timeline, pos_h, 'b')
axs.plot(timeline, pos_w, 'r')
axs.axis('tight')

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(15, 15))
axs.plot(pos_true, pos_h, 'b')
axs.plot(pos_true, pos_w, 'r')
axs.axis('tight')

Wow!